# FAQ recommendations

This Jupyter Notebook conducts an evaluation of various approaches to develop an LLM algorithm capable of recommending solutions based on an F.A.Q. database and the conversation context between an agent and a client.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 1 handling the data

## 1.1.0 Converting F.A.Q from .txt to .csv file

In [2]:
def get_questions_and_answers(faq_path: str):
    questions = []
    answers = []

    with open(faq_path, "r") as file:
        for line in file.readlines():
            if "Pergunta" in line:
                question = line.lstrip("Pergunta: ").rstrip("\n")
                questions.append(question)
            elif "Resposta" in line:
                answer = line.lstrip("Resposta: ").rstrip("\n")
                answers.append(answer)
    
    return (questions, answers)


def create_csv(questions: list, answers: list, destination_path: str, file_name: str):
    faq_dict = {
        "Question": questions,
        "Answer": answers
        }
    
    path = destination_path + file_name + ".csv"

    df = pd.DataFrame(data=faq_dict, dtype=object)
    df.to_csv(path_or_buf=path, sep=";", index=False)

In [3]:
questions, answers = get_questions_and_answers(faq_path="../data/raw/FAQ_example.txt")
create_csv(questions=questions, answers=answers, destination_path="../data/raw/", file_name="FAQ_example")

# 2 Testing LLMs techniques

## 2.1 Sentence Embeddings pretrained models

In [4]:
from sentence_transformers import SentenceTransformer, util
from spellchecker import SpellChecker


def get_greatest_similarities(model, dataframe, message):
    questions_embeddings = model.encode(dataframe.Question)
    message_embed = model.encode(message)

    similarities = util.cos_sim(message_embed, questions_embeddings)[0].tolist()
    greatest_similarities_index = pd.Series(similarities).sort_values(ascending=False).index
    
    return greatest_similarities_index

c:\Users\rafae\miniconda3\envs\techlab\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
df = pd.read_csv("../data/raw/FAQ_example.csv", sep=";")
df.head(3)

,Question,Answer
0,Quais serviços a empresa de telecomunicações o...,Nossa empresa oferece serviços de telefonia fi...
1,Como posso contratar os serviços da empresa?,"Para contratar nossos serviços, você pode entr..."
2,Quais são os planos disponíveis para a telefon...,Temos diversos planos com diferentes franquias...


In [7]:
message = "é possível melhorar meu plano?"

### 2.1.1 all-mpnet-base-v2 model

In [8]:
model = SentenceTransformer("all-mpnet-base-v2")

print(message)
df.iloc[get_greatest_similarities(model, df, message)].head(10)

é possível melhorar meu plano?


,Question,Answer
9,Como posso efetuar o pagamento da minha fatura?,"Aceitamos diversos métodos de pagamento, inclu..."
2,Quais são os planos disponíveis para a telefon...,Temos diversos planos com diferentes franquias...
10,É possível fazer upgrade ou downgrade nos plan...,"Sim, você pode fazer o upgrade ou downgrade do..."
5,É possível fazer a portabilidade do meu número...,"Sim, é possível fazer a portabilidade do seu n..."
3,A empresa oferece planos empresariais?,"Sim, temos planos especiais para empresas, com..."
13,Como posso acompanhar meu consumo de internet ...,Você pode acompanhar seu consumo através do no...
27,Como posso acompanhar o status de uma solicita...,Você pode acompanhar o status de suas solicita...
19,A empresa oferece planos para clientes que uti...,"Sim, temos planos com franquias de dados maior..."
1,Como posso contratar os serviços da empresa?,"Para contratar nossos serviços, você pode entr..."
7,A empresa oferece suporte técnico?,"Sim, nossa empresa oferece suporte técnico esp..."


### 2.1.2 multi-qa-mpnet-base-dot-v1

In [9]:
model = SentenceTransformer("multi-qa-mpnet-base-dot-v1")

print(message)
df.iloc[get_greatest_similarities(model, df, message)].head(10)

é possível melhorar meu plano?


,Question,Answer
3,A empresa oferece planos empresariais?,"Sim, temos planos especiais para empresas, com..."
2,Quais são os planos disponíveis para a telefon...,Temos diversos planos com diferentes franquias...
9,Como posso efetuar o pagamento da minha fatura?,"Aceitamos diversos métodos de pagamento, inclu..."
19,A empresa oferece planos para clientes que uti...,"Sim, temos planos com franquias de dados maior..."
10,É possível fazer upgrade ou downgrade nos plan...,"Sim, você pode fazer o upgrade ou downgrade do..."
5,É possível fazer a portabilidade do meu número...,"Sim, é possível fazer a portabilidade do seu n..."
7,A empresa oferece suporte técnico?,"Sim, nossa empresa oferece suporte técnico esp..."
16,A empresa oferece atendimento em outros idioma...,"No momento, nosso atendimento é realizado apen..."
6,Como posso verificar a cobertura de internet n...,Para verificar a cobertura de internet em sua ...
13,Como posso acompanhar meu consumo de internet ...,Você pode acompanhar seu consumo através do no...


### 2.1.3 all-distilroberta-v1 model

In [10]:
model = SentenceTransformer("all-distilroberta-v1")

print(message)
df.iloc[get_greatest_similarities(model, df, message)].head(10)

é possível melhorar meu plano?


,Question,Answer
5,É possível fazer a portabilidade do meu número...,"Sim, é possível fazer a portabilidade do seu n..."
2,Quais são os planos disponíveis para a telefon...,Temos diversos planos com diferentes franquias...
19,A empresa oferece planos para clientes que uti...,"Sim, temos planos com franquias de dados maior..."
10,É possível fazer upgrade ou downgrade nos plan...,"Sim, você pode fazer o upgrade ou downgrade do..."
18,É possível fazer a rescisão dos serviços a qua...,"Sim, você pode solicitar a rescisão dos serviç..."
9,Como posso efetuar o pagamento da minha fatura?,"Aceitamos diversos métodos de pagamento, inclu..."
3,A empresa oferece planos empresariais?,"Sim, temos planos especiais para empresas, com..."
1,Como posso contratar os serviços da empresa?,"Para contratar nossos serviços, você pode entr..."
27,Como posso acompanhar o status de uma solicita...,Você pode acompanhar o status de suas solicita...
13,Como posso acompanhar meu consumo de internet ...,Você pode acompanhar seu consumo através do no...


### 2.1.4 all-MiniLM-L6-v2 model

In [11]:
model = SentenceTransformer("all-MiniLM-L6-v2")

print(message)
df.iloc[get_greatest_similarities(model, df, message)].head(10)

é possível melhorar meu plano?


,Question,Answer
5,É possível fazer a portabilidade do meu número...,"Sim, é possível fazer a portabilidade do seu n..."
18,É possível fazer a rescisão dos serviços a qua...,"Sim, você pode solicitar a rescisão dos serviç..."
2,Quais são os planos disponíveis para a telefon...,Temos diversos planos com diferentes franquias...
19,A empresa oferece planos para clientes que uti...,"Sim, temos planos com franquias de dados maior..."
27,Como posso acompanhar o status de uma solicita...,Você pode acompanhar o status de suas solicita...
24,Como posso agendar a visita de um técnico para...,Você pode agendar a visita de um técnico entra...
7,A empresa oferece suporte técnico?,"Sim, nossa empresa oferece suporte técnico esp..."
3,A empresa oferece planos empresariais?,"Sim, temos planos especiais para empresas, com..."
10,É possível fazer upgrade ou downgrade nos plan...,"Sim, você pode fazer o upgrade ou downgrade do..."
9,Como posso efetuar o pagamento da minha fatura?,"Aceitamos diversos métodos de pagamento, inclu..."


### 2.1.5 multi-qa-MiniLM-L6-cos-v1

In [12]:
model = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1")

print(message)
df.iloc[get_greatest_similarities(model, df, message)].head(10)

é possível melhorar meu plano?


,Question,Answer
3,A empresa oferece planos empresariais?,"Sim, temos planos especiais para empresas, com..."
19,A empresa oferece planos para clientes que uti...,"Sim, temos planos com franquias de dados maior..."
5,É possível fazer a portabilidade do meu número...,"Sim, é possível fazer a portabilidade do seu n..."
7,A empresa oferece suporte técnico?,"Sim, nossa empresa oferece suporte técnico esp..."
2,Quais são os planos disponíveis para a telefon...,Temos diversos planos com diferentes franquias...
9,Como posso efetuar o pagamento da minha fatura?,"Aceitamos diversos métodos de pagamento, inclu..."
18,É possível fazer a rescisão dos serviços a qua...,"Sim, você pode solicitar a rescisão dos serviç..."
24,Como posso agendar a visita de um técnico para...,Você pode agendar a visita de um técnico entra...
27,Como posso acompanhar o status de uma solicita...,Você pode acompanhar o status de suas solicita...
10,É possível fazer upgrade ou downgrade nos plan...,"Sim, você pode fazer o upgrade ou downgrade do..."


### 2.1.6 msmarco-distilbert-base-v4

In [13]:
model = SentenceTransformer("msmarco-distilbert-base-v4")

print(message)
df.iloc[get_greatest_similarities(model, df, message)].head(10)

é possível melhorar meu plano?


,Question,Answer
5,É possível fazer a portabilidade do meu número...,"Sim, é possível fazer a portabilidade do seu n..."
18,É possível fazer a rescisão dos serviços a qua...,"Sim, você pode solicitar a rescisão dos serviç..."
1,Como posso contratar os serviços da empresa?,"Para contratar nossos serviços, você pode entr..."
10,É possível fazer upgrade ou downgrade nos plan...,"Sim, você pode fazer o upgrade ou downgrade do..."
9,Como posso efetuar o pagamento da minha fatura?,"Aceitamos diversos métodos de pagamento, inclu..."
3,A empresa oferece planos empresariais?,"Sim, temos planos especiais para empresas, com..."
13,Como posso acompanhar meu consumo de internet ...,Você pode acompanhar seu consumo através do no...
22,É possível obter um número de telefone fixo em...,"Sim, você pode contratar um número de telefone..."
2,Quais são os planos disponíveis para a telefon...,Temos diversos planos com diferentes franquias...
27,Como posso acompanhar o status de uma solicita...,Você pode acompanhar o status de suas solicita...
